In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
import calendar
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from pycaret.classification import *

In [2]:
pd.get_option("display.max_columns", None)
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

In [5]:
Load_10K_Records=True
currency_pairs = ["EUR_USD"]
timeframe = "H4"
price_char = "M"
price_com = "mid"

In [6]:
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}

In [7]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

In [8]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [9]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)

In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

## Simple Moving Average (SMA)

In [15]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(5)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(5)

## Moving Average Range

In [16]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']

## Price Range

In [17]:
data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

In [18]:
data = data.drop_duplicates()
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [19]:
data.tail(3)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,F_SMA_5,F_SMA_10,col_1,col_2,col_3
9647,2021-12-13,18:00:00,18,347,0,Monday,2021-12-13T18:00:00.000000000Z,5485,1.12933,1.12999,1.12818,1.12862,1.12859,1.12948,0.00003,-0.00086,0.00071,0.00066,0.00044
9648,2021-12-13,22:00:00,22,347,0,Monday,2021-12-13T22:00:00.000000000Z,3039,1.12844,1.12850,1.12738,1.12824,1.12829,1.12948,-0.00005,-0.00124,0.00020,0.00006,0.00086
9649,2021-12-14,02:00:00,2,348,1,Tuesday,2021-12-14T02:00:00.000000000Z,2859,1.12822,1.12838,1.12716,1.12774,1.12847,1.12932,-0.00073,-0.00158,0.00048,0.00016,0.00058


## Candlestick Number

In [20]:
candle_no = len(data) - 2
#candle_no = 9659

In [21]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.show()

In [22]:
candle = data.iloc[candle_no-3:candle_no+1]

In [23]:
candle[['Volume']]

,Volume
9645,13558
9646,19274
9647,5485
9648,3039


## Average True Range (ATR)

In [24]:
high_low = data['High'] - data['Low']
high_cp = np.abs(data['High'] - data['Close'].shift())
low_cp = np.abs(data['Low'] - data['Close'].shift())
df = pd.concat([high_low, high_cp, low_cp], axis=1)
true_range = np.max(df, axis=1)
data['ATR_14'] = true_range.rolling(14).mean()

## Stop Loss / TakeProfit

In [25]:
ATR = data.iloc[candle_no]['ATR_14']
CLOSED_PRICE = data.iloc[candle_no]['Close']
BUY_SL = (CLOSED_PRICE - ATR).round(5)
SELL_SL = (CLOSED_PRICE + ATR).round(5)
BUY_TP = (CLOSED_PRICE + ATR).round(5)
SELL_TP = (CLOSED_PRICE - ATR).round(5)

## Feature Selection

In [26]:
data = data[['col_1','col_2','col_3','F_SMA_5','F_SMA_10']]

In [27]:
data.head(1)

,col_1,col_2,col_3,F_SMA_5,F_SMA_10
0,0.00215,0.00167,0.00113,-0.00312,-0.00084


## Model

In [28]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
        else:
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]]['col_1'],
                            dataset.iloc[indices.flatten()[i]]['col_2'],
                            dataset.iloc[indices.flatten()[i]]['col_3'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      'col_1',
                      'col_2',
                      'col_3',
                      'F_SMA_5',
                      'F_SMA_10',
                     ]
    
    return indices, distances

In [29]:
indices, distances = find_k_similar_candles (candle_no,data)

,col_1,col_2,col_3,F_SMA_5,F_SMA_10
9648,0.0002,0.00006,0.00086,-0.00005,-0.00124


## Final Action

In [30]:
indices = indices[0:1][0]

In [31]:
recs = []

for indice in indices[1:10]:
    
    data = pd.read_csv(filename) 
    data = data.iloc[indice:indice+3]
    
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X)
    
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        recs.append((r2_score(Y, y_pred).round(2)*100))
    else:
        recs.append((r2_score(Y, y_pred).round(2)*100) * -1)

In [32]:
data_unseen = pd.DataFrame ({'Rec1': [recs[0]], 'Rec2': [recs[1]], 'Rec3':[recs[2]]})

In [33]:
SAVED_FINAL_MODEL = load_model('EURUSD/FINAL_LIGHTGBM_17Nov2021_EURUSD')
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)
new_prediction.head()

Transformation Pipeline and Model Successfully Loaded


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [ ]:
LIGHTGBM_Pre = new_prediction['Label']
LIGHTGBM_Pre[0]

In [ ]:
SAVED_FINAL_MODEL = load_model('EURUSD/FINAL_GBC_17Nov2021_EURUSD')
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)
new_prediction.head()

In [ ]:
LIGHTGBM_Pre[0]

In [ ]:
BUY_SL

In [ ]:
BUY_TP

In [ ]:
SELL_SL

In [ ]:
SELL_TP